In [1]:
!pip install transformers

In [2]:
import torch
import pandas as pd
from torch import nn
# from transformers import AutoModel, ElectraConfig
# from transformers import ElectraTokenizer
from transformers import BertTokenizer, BertModel

# ELECTRA_TOKENIZER = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
BERT_TOKENIZER = BertTokenizer.from_pretrained('bert-base-uncased')

# ELECTRA model with custom output layer.
# class Electra(nn.Module):
#     def __init__(self, output_size, device='cpu'):
#         super().__init__()
#         self.device = device
#         config = ElectraConfig.from_pretrained('google/electra-small-discriminator')
#         self.model = AutoModel.from_config(config).to(device)
#         self.output = nn.Linear(self.model.config.hidden_size, output_size).to(device)

#     # What happens when passing input into the model.
#     def forward(self, sents, locs):
#         # sents = torch.tensor(sents).to(self.device)
#         # print(self.electra(sents))
#         sents = self.model(sents)[0]
#         abbs = torch.stack([sents[n, idx, :] for n, idx in enumerate(locs)])  # (B * M)
#         return self.output(abbs)

class Bert(nn.Module):
    def __init__(self, output_size, device='cpu'):
        super().__init__()
        self.device = device
        self.model = BertModel.from_pretrained('bert-base-uncased').to(device)
        self.output = nn.Linear(self.model.config.hidden_size, output_size).to(device)

    # What happens when passing input into the model.
    def forward(self, sents, locs):
        # sents = torch.tensor(sents).to(self.device)
        # print(self.electra(sents))
        sents = self.model(sents)[0]
        abbs = torch.stack([sents[n, idx, :] for n, idx in enumerate(locs)])  # (B * M)
        return self.output(abbs)

In [3]:
class MedalDatasetTokenizer(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, dictionary_file, max_length=256, device='cpu'):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.device = device
        self.df = df
        label_df = pd.read_csv(dictionary_file, sep='\t', index_col = "EXPANSION")
        self.label_ser = label_df["LABEL"].squeeze()


    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idxs):
        
        # Code to remove entries that are larger than the max length size
        # batch_df = self.df.iloc[idxs]
        # # ic(batch_df['TEXT'].apply(lambda string: len(string.split())))
        # filter = batch_df['TEXT'].apply(lambda string: len(string.split()) < self.max_length).to_list()
        # # ic(idxs, filter)
        # idxs = list(compress(idxs, filter))


        batch_df = self.df.iloc[idxs]
        locs = batch_df['LOCATION'].values
        label_strings = batch_df['LABEL'].values
        labels = self.label_ser[label_strings].to_numpy()

        # ic(batch_df['TEXT'].tolist())
        # ic(type(batch_df['TEXT'].tolist()[0]))
        batch_encode = self.tokenizer(batch_df['TEXT'].tolist(), max_length=self.max_length, \
                    padding=True, truncation = True)
        
        # ic(batch_encode)
        # ic(type(batch_encode))

        tokenized = batch_encode['input_ids']
        # decoded = self.tokenizer.batch_decode(tokenized)
        # ic(decoded, len(decoded[0].split()))
        # ic(len(tokenized[0]), len(tokenized[1]), type(tokenized))
        return torch.tensor(tokenized).to(self.device), torch.tensor(locs).to(self.device), torch.tensor(labels).to(self.device)


In [4]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
from datetime import datetime
import os
from time import time 

In [5]:
def train_loop(train_data, model, loss_fn, optimizer, train_loader, max = -1):
    
    size = len(train_data)

    # Switches model to training mode.
    model.train()
    
    # List of all values for the loss. Output at the end.
    loss_list = []
    # For computing accuracy
    correct = 0

    for batch, idx in enumerate(tqdm(train_loader)):
    # for batch, idx in enumerate(train_loader):
        # print(idx)
        X = train_data[idx][0]
        loc = train_data[idx][1]
        y = train_data[idx][2]

        # Compute prediction and loss
        pred = model(X, loc)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Record the loss and accuracy
        loss_value = loss.item()
        batch_correct = (pred.argmax(1) == y).type(torch.float).sum().item()
        correct += batch_correct
        loss_list.append(loss_value)

        # Terminate early for testing purposes
        if max > 0:
            if batch >= max:
                print("\nMax iterations reached.")
                break

        # Minibatch loss
        if batch % 20 == 0 and batch != 0:
            print(f"\nBatch loss: {loss_value:>7f}")
    
    loss_list = np.array(loss_list)
    mean_loss = np.mean(loss_list)
    accuracy = correct/size
    print(f"Accuracy: {accuracy:>3f} | Average Loss: {mean_loss:>7f}\n")
    return mean_loss, accuracy

# Tests the model on the validation data
def valid_loop(valid_data, model, loss_fn, valid_loader, max = -1):

    # Switches model to evaluation mode
    model.eval()

    size = len(valid_data)
    loss_list = [] 
    correct = 0

    with torch.no_grad():
        for batch, idx in tqdm(enumerate(valid_loader)):
        # for batch, idx in enumerate(valid_loader):
            # idx = torch.tensor([id])
            # print(id, idx)
            X = valid_data[idx][0]
            loc = valid_data[idx][1]
            y = valid_data[idx][2]
            pred = model(X, loc)
            loss_list.append(loss_fn(pred, y).item())
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            if max > 0:
                if batch > max:
                    break

    valid_loss = np.mean(np.array(loss_list))
    correct /= size
    print(f"Validation| \nAccuracy: {correct:>3f} | Average loss: {valid_loss:>8f} \n")
    return valid_loss, correct

In [6]:
num_abbr = "two_abbr"
folder = "drive/MyDrive/Bootcamp"

In [7]:
def save_model(model, save_dir):
    now = datetime.now()
    now_formatted = now.strftime("%d")+"_"+now.strftime("%H")+"_"+now.strftime("%M")
    torch.save(model, save_dir + f"{now_formatted}_two_abbr_Electra.pt")
    print("Model saved\n")

In [8]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu" 
device = torch.device(dev) 

# N_CPU_CORES = 2
# torch.set_num_threads(N_CPU_CORES)

# tokenizer = ELECTRA_TOKENIZER
tokenizer = BERT_TOKENIZER
max = -1

# Data
num_abbr = "two_abbr"
# folder = "datasets/medal"
train_df = pd.read_csv(f"{folder}/{num_abbr}/train.csv")
dictionary_file = f"{folder}/{num_abbr}/dict.txt"
output_size = 25 # Should be set to the size of the dictionary
train_data = MedalDatasetTokenizer(train_df, tokenizer, dictionary_file, device = device)

valid_df = pd.read_csv(f"{folder}/{num_abbr}/valid.csv")
valid_data = MedalDatasetTokenizer(valid_df, tokenizer, dictionary_file, device = device)

# Hyperparameters
learning_rate = 4e-5
batch_size = 16
epochs = 10

# Model
model = Bert(output_size, device)
#Electra(output_size=output_size, device=device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()


# Train the model
for t in range(epochs):
    print(f"\nEpoch {t+1}\n-------------------------------")
    
    train_loader = DataLoader(
        range(len(train_data)), 
        shuffle=True, 
        batch_size=batch_size
    )

    valid_loader = DataLoader(
        range(len(valid_data)), 
        shuffle=True, 
        batch_size=batch_size
    )

    start = time()
    train_loss, train_accuracy = train_loop(train_data, model, loss_fn, optimizer, train_loader, max = max)
    end = time()
    print(f"Training time: {end-start}\n")

    valid_loss, valid_accuracy = valid_loop(valid_data, model, loss_fn, valid_loader, max = max)

    with open(f"{folder}/saves/loss.txt", "a") as file:
        file.writelines(f"\n{t+1},{train_loss},{train_accuracy},{valid_loss},{valid_accuracy}")

    save_model(model, f"{folder}/saves/{num_abbr}_epoch{t}")



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/271 [00:00<?, ?it/s]


Epoch 1
-------------------------------


  8%|▊         | 21/271 [00:19<03:58,  1.05it/s]


Batch loss: 3.310792


 15%|█▌        | 41/271 [00:39<03:36,  1.06it/s]

...


Batch loss: 0.006113


100%|██████████| 271/271 [04:10<00:00,  1.08it/s]
0it [00:00, ?it/s]

Accuracy: 0.985651 | Average Loss: 0.050514

Training time: 250.39060235023499



89it [00:37,  2.37it/s]


Validation| 
Accuracy: 0.870922 | Average loss: 0.590501 

Model saved

